# AMANI × MedGemma — Multilingual Medical Resource Routing

**Kaggle AI in Medicine Hackathon 2026**

This notebook demonstrates the full AMANI pipeline: a 5-layer AI architecture that routes complex international patients to appropriate clinical trials, regenerative medicine programs, and frontier medical technologies.

**Powered by:** Google MedGemma 1.5 4B — a medically-specialized vision-language model trained on healthcare data.

---

## Architecture Overview

```
Clinical Note (ZH/AR/TH/EN)
        ↓
  L1 Sentinel Gate    ← Entropy-based quality filter (D-value)
        ↓
  L2 MedGemma         ← Clinical parsing + urgency detection
  L2 Trinity-Audit    ← 3-model consensus (MedGemma + GPT-4o + Claude)
  L2 TrialMatching    ← Semantic keyword-overlap trial scoring
  L2 AssetResolution  ← AGID registry lookup
        ↓
  L2.5 Value/TDLS     ← Total Disease Lifecycle Strategy + Shadow Quote
        ↓
  L3 Nexus Router     ← Global institution + travel routing
        ↓
  Structured Output   ← Trial match, cost, timeline, AGID
```

---

## Step 1 — Install Dependencies

In [ ]:
# Install required packages
# MedGemma requires HuggingFace access token (set HUGGING_FACE_HUB_TOKEN)
# In mock mode (default), no GPU or token needed
!pip install gradio>=4.0 transformers torch accelerate --quiet

## Step 2 — Clone Repository

In [ ]:
import os

# Clone the AMANI-MedGemma repository
# Replace with your actual Kaggle/GitHub repo URL after submission
# !git clone https://github.com/your-username/amani-medgemma.git
# %cd amani-medgemma

# For Kaggle: files are already in /kaggle/working/
# Uncomment if running on Kaggle:
# %cd /kaggle/working/amani-medgemma

print('Repository ready. Running in:', os.getcwd())

## Step 3 — Configure Environment

By default the pipeline runs in **mock mode** — deterministic demo outputs requiring no GPU or API keys.

To use real MedGemma inference, set `AMANI_MOCK_MODE=false` and provide a HuggingFace token.

In [ ]:
import os

# --- MODE SELECTION ---
# 'true'  = Mock mode (fast, no GPU required, deterministic demo outputs)
# 'false' = Real MedGemma 1.5 4B inference (requires GPU + HuggingFace token)
os.environ['AMANI_MOCK_MODE'] = 'true'

# Optional: set HuggingFace token for real inference
# os.environ['HUGGING_FACE_HUB_TOKEN'] = 'hf_...'

print(f"Mock mode: {os.environ['AMANI_MOCK_MODE']}")
print("Pipeline configured.")

## Step 4 — Import Pipeline

In [ ]:
import sys
import json

# Add project root to path if needed
# sys.path.insert(0, '/path/to/amani-medgemma')

from app import run_full_pipeline

print('AMANI pipeline imported successfully.')

## Step 5 — Case A: Chinese NSCLC Patient (Mandarin)

**Patient:** 52-year-old male, Non-Small Cell Lung Cancer Stage IIIB, EGFR L858R+, third-line failure. Seeking CAR-T or gene therapy clinical trial.

**Input language:** Simplified Chinese (ZH)

In [ ]:
note_a = "患者男性，52岁，非小细胞肺癌IIIB期。EGFR L858R阳性。三线治疗后进展。寻求基因治疗或CAR-T临床试验。"

print("Processing Case A (ZH — NSCLC IIIB)...")
result_a = run_full_pipeline(note_a, 'case_a')

# --- Display key outputs ---
s = result_a.get('summary', {})
L = result_a['layers']

print(f"\n{'='*60}")
print("CASE A RESULTS")
print(f"{'='*60}")
print(f"  D-value (precision):  {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:              {L['L2_MedGemma']['urgency']}")
print(f"  Trinity consensus:    {L['L2_Trinity']['status']} → {L['L2_Trinity']['consensus_agid']}")

tm = L.get('L2_TrialMatching', {})
if tm.get('top_matches'):
    top = tm['top_matches'][0]
    print(f"  Top trial:            {top['nct_id']} (score={top['match_score']})")
    print(f"  Institution:          {top['institution']}")
    print(f"  Trial PI:             {top['pi']}")

tdls = L.get('L2_5_Value', {})
print(f"  Total pathway cost:   ${tdls.get('total_cost_usd', 0):,.0f}")
print(f"  Total duration:       {tdls.get('total_duration_days', 0)} days")

nexus = L.get('L3_Nexus', {})
print(f"  Routing:              {nexus.get('routing_decision', 'N/A')}")
print(f"  Destination:          {nexus.get('destination_country', 'N/A')}")

## Step 6 — Case B: Saudi Stem Cell Patient (English/Arabic context)

**Patient:** 68-year-old Saudi male, post-CABG 2019, stable CAD, bilateral knee OA Grade III. MoCA 26/30. Seeking comprehensive stem cell regenerative program in Japan.

**Input language:** English

In [ ]:
note_b = ("68-year-old Saudi male, post-CABG 2019, stable CAD, bilateral knee OA Grade III. "
           "MoCA 26/30. Seeking comprehensive stem cell regenerative program in Japan.")

print("Processing Case B (EN — Stem Cell / Regenerative Medicine)...")
result_b = run_full_pipeline(note_b, 'case_b')

L = result_b['layers']
tm = L.get('L2_TrialMatching', {})
tdls = L.get('L2_5_Value', {})

print(f"\n{'='*60}")
print("CASE B RESULTS")
print(f"{'='*60}")
print(f"  D-value (precision):  {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:              {L['L2_MedGemma']['urgency']} (elective → timeline ×1.3)")
print(f"  Trinity consensus:    {L['L2_Trinity']['status']} → {L['L2_Trinity']['consensus_agid']}")

if tm.get('top_matches'):
    top = tm['top_matches'][0]
    print(f"  Top trial:            {top['nct_id']} (score={top['match_score']})")
    print(f"  Institution:          {top['institution']}")

print(f"  Total pathway cost:   ${tdls.get('total_cost_usd', 0):,.0f}")
print(f"  Total duration:       {tdls.get('total_duration_days', 0)} days (elective-extended)")

# TDLS stage breakdown
print(f"\n  TDLS Stage Breakdown:")
for stage in tdls.get('stages', []):
    print(f"    Stage {stage['stage']}: {stage['title'][:45]:<45} ${stage['cost_usd']:>8,.0f}  {stage['duration_days']}d")

## Step 7 — Case C: Thai Parkinson's Patient (Thai language)

**Patient:** 61-year-old Thai male, Parkinson's H&Y Stage 4, post bilateral STN-DBS 2022 (declining benefit). Seeking BCI clinical trial in the United States.

**Input language:** Thai (TH)

In [ ]:
note_c = "ผู้ป่วยชายไทย อายุ 61 ปี โรคพาร์กินสัน H&Y Stage 4 ได้รับการผ่าตัด DBS ปี 2022 ต้องการเข้าถึง BCI clinical trial ในสหรัฐอเมริกา"

print("Processing Case C (TH — Parkinson's BCI Trial)...")
result_c = run_full_pipeline(note_c, 'case_c')

L = result_c['layers']
tm = L.get('L2_TrialMatching', {})
tdls = L.get('L2_5_Value', {})

print(f"\n{'='*60}")
print("CASE C RESULTS")
print(f"{'='*60}")
print(f"  D-value (precision):  {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:              {L['L2_MedGemma']['urgency']}")
print(f"  Trinity consensus:    {L['L2_Trinity']['status']} → {L['L2_Trinity']['consensus_agid']}")
print(f"  Matches found:        {tm.get('matches_found', 0)}")

if tm.get('top_matches'):
    for i, m in enumerate(tm['top_matches'][:2]):
        print(f"  Trial #{i+1}:             {m['nct_id']} (score={m['match_score']}) — {m['institution']}")

print(f"  Total pathway cost:   ${tdls.get('total_cost_usd', 0):,.0f}")
print(f"  Total duration:       {tdls.get('total_duration_days', 0)} days")

## Step 8 — Cross-Case Comparison

Demonstrating that the pipeline produces **differentiated, case-specific outputs** — not generic responses.

In [ ]:
print(f"{'='*70}")
print(f"{'CROSS-CASE COMPARISON':^70}")
print(f"{'='*70}")
print(f"{'Metric':<30} {'Case A (ZH/NSCLC)':<20} {'Case B (EN/StemCell)':<22} {'Case C (TH/PD)'}")
print(f"{'-'*70}")

cases = [
    ('result_a', result_a),
    ('result_b', result_b),
    ('result_c', result_c),
]

# Extract metrics
metrics = {}
for key, r in cases:
    L = r['layers']
    tm = L.get('L2_TrialMatching', {})
    top_score = tm['top_matches'][0]['match_score'] if tm.get('top_matches') else 0
    metrics[key] = {
        'd_value': L['L1_Sentinel']['d_value'],
        'urgency': L['L2_MedGemma']['urgency'],
        'agid': L['L2_Trinity']['consensus_agid'],
        'trial_score': top_score,
        'cost': L.get('L2_5_Value', {}).get('total_cost_usd', 0),
        'days': L.get('L2_5_Value', {}).get('total_duration_days', 0),
    }

rows = [
    ('D-value', 'd_value', '{:.3f}'),
    ('Urgency', 'urgency', '{}'),
    ('Top AGID', 'agid', '{}'),
    ('Trial score', 'trial_score', '{:.2f}'),
    ('Cost (USD)', 'cost', '${:,.0f}'),
    ('Duration (days)', 'days', '{}'),
]

for label, key, fmt in rows:
    vals = [fmt.format(metrics[k][key]) for k in ['result_a', 'result_b', 'result_c']]
    print(f"{label:<30} {vals[0]:<20} {vals[1]:<22} {vals[2]}")

print(f"{'='*70}")
print("\nScore differentiation confirmed:", 
      len(set(metrics[k]['trial_score'] for k in metrics)) >= 2)

## Step 9 — Noise Input Safety Test

Demonstrating L1 Sentinel gate and Trinity SOFT_CONFLICT handling for non-medical queries.

In [ ]:
noise_input = "What is the weather today?"

print("Testing noise input handling...")
result_noise = run_full_pipeline(noise_input, 'auto')

L = result_noise['layers']
print(f"\n  Input:          '{noise_input}'")
print(f"  L1 status:      {L['L1_Sentinel']['status']}")
print(f"  Trinity status: {L.get('L2_Trinity', {}).get('status', 'N/A')}")
print(f"  AGID:           {L.get('L2_Trinity', {}).get('consensus_agid', 'N/A')}")
print(f"  Warning:        {result_noise.get('warning', 'None')}")
print("\n  Graceful degradation confirmed — no crash, appropriate routing to HITL.")

## Step 10 — Launch Gradio UI

Interactive 4-tab interface for exploring all pipeline layers.

In [ ]:
from app import build_gradio_app

ui = build_gradio_app()

# Launch the Gradio interface
# share=True creates a public link (useful for Kaggle/Colab)
ui.launch(share=True)

---

## Architecture Credits

| Component | Technology |
|-----------|------------|
| Medical AI core | Google MedGemma 1.5 4B (`google/medgemma-1.5-4b-it`) |
| Trinity consensus | MedGemma + GPT-4o + Claude (3-model audit) |
| UI framework | Gradio 4.x |
| Trial database | Demo: NCT-06234517, NCT-06578901, NCT-06891023 |
| AGID registry | AMANI Asset Graph ID system |
| Languages | ZH / AR / TH / EN |

**License:** Apache 2.0  
**Submission:** Kaggle AI in Medicine Hackathon 2026  
**Architecture:** AMANI Sovereign Medical Routing Protocol